### Tutorial: How to Use the Given Code

This notebook is a guide to using the code provided in our package. The first thing we would need to do is to download the package onto your computer. After this, you can simply import the package using the command:

In [6]:
%load_ext autoreload
%autoreload 2
import os, sys
import MCAT_pkg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


After we import our package, we will need to load in our data. 

In [ ]:
# Identify the location that the data file is in 
data_path = "../data/" 

# Read in the file as df
df = pd.read_csv(os.path.join(data_path, "gardner_mt_catastrophe_only_tubulin.csv"), comment = "#")